In [7]:
import logging

import nshutils

nshutils.pretty(rich_log_handler=False)
logging.basicConfig(level=logging.INFO)

In [2]:
from jmppeft.utils import wbm_relax

ctx = wbm_relax.eval_context()
ctx.__enter__()

Type checking the following modules: ('jmppeft',)


Loading 'wbm_summary' from cached file at '/root/.cache/matbench-discovery/1.0.0/wbm/2023-12-13-wbm-summary.csv.gz'


In [3]:
from pathlib import Path

import rich

base_ckpt = Path("/mnt/datasets/jmp-mptrj-checkpoints/sm_checkpoints/")
best_ckpts: dict[str, Path] = {}
for ckpt in base_ckpt.glob("*/checkpoint/last.ckpt"):
    best_ckpts[ckpt.parent.parent.name] = ckpt.resolve()
rich.print(best_ckpts)

{
    'mptrj-jmps-ec20.0_fc20.0_sc10.0': 
Path('/mnt/datasets/jmp-mptrj-checkpoints/sm_checkpoints/mptrj-jmps-ec20.0_fc20.0_sc10.0/checkpoint/epoch=139-step=
396200-val_matbench_discovery_force_mae=0.01921326294541359.ckpt'),
    'mptrj-jmps-s2efonly': 
Path('/mnt/datasets/jmp-mptrj-checkpoints/sm_checkpoints/mptrj-jmps-s2efonly/checkpoint/epoch=129-step=367900-val_m
atbench_discovery_force_mae=0.018848825246095657.ckpt'),
    'mptrj-jmps-s2ef_s2re': 
Path('/mnt/datasets/jmp-mptrj-checkpoints/sm_checkpoints/mptrj-jmps-s2ef_s2re/checkpoint/epoch=125-step=356580-val_
matbench_discovery_force_mae=0.01882246695458889.ckpt')
}

In [4]:
import copy
from typing import Any

import numpy as np
import torch

setup = wbm_relax.RelaxSetup()
setup.device = torch.device("cuda:0")
setup.dtype = torch.float32
setup.relax_config.compute_stress = True
setup.relax_config.stress_weight = 0.1
setup.relax_config.optimizer = "FIRE"
setup.relax_config.fmax = 0.05
setup.relax_config.ase_filter = "exp"
setup.linref = np.load("/workspaces/repositories/jmp-peft/notebooks/mptrj_linref.npy")


def update_hparams(hparams: dict[str, Any]):
    hparams = copy.deepcopy(hparams)
    hparams.pop("environment", None)
    hparams.pop("trainer", None)
    hparams.pop("runner", None)
    hparams.pop("directory", None)
    hparams.pop("ckpt_load", None)
    return hparams


model = wbm_relax.load_ckpt(
    best_ckpts["mptrj-jmps-ec20.0_fc20.0_sc10.0"],
    setup=setup,
    update_hparams=update_hparams,
)
model

/opt/conda/envs/jmp-peft/lib/python3.11/site-packages/lightning/pytorch/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.3.1, which is newer than your current Lightning version: v2.2.1
NSHRUNNER_SESSION_DIR is not set. Skipping symlink creation.
NSHRUNNER_SESSION_DIR is not set. Skipping symlink creation.
NSHRUNNER_SESSION_DIR is not set. Skipping symlink creation.
NSHRUNNER_SESSION_DIR is not set. Skipping symlink creation.
Using regular backbone


Unrecognized arguments:  dict_keys(['name', 'learnable_rbf', 'learnable_rbf_stds', 'unique_basis_per_layer', 'dropout', 'edge_dropout', 'ln_per_layer', 'scale_factor_to_ln'])


Constructed backbone with dlora=None
Freezing 0 parameters (0.00%) out of 43,146,824 total parameters (43,146,824 trainable)


MatbenchDiscoveryModel(config=MatbenchDiscoveryConfig(name=mptrj-jmps-ec20.0_fc20.0_sc10.0, project=jmp_mptrj), device=cuda:0)

In [13]:
dl = wbm_relax.setup_dataset_and_loader(num_items=16384, model=model, setup=setup)
print(len(dl))

16384


In [14]:
preds_targets = wbm_relax.relax_loop(dl, setup=setup, model=model)
print(len(preds_targets))

  0%|          | 0/16384 [00:00<?, ?it/s]

INFO:jmppeft.utils.wbm_relax:# Steps: 24; e_form: P=-3.1896, GT=-3.2065, Δ=0.0169, MAE=0.0169
INFO:jmppeft.utils.wbm_relax:# Steps: 19; e_form: P=0.0609, GT=0.0921, Δ=0.0313, MAE=0.0241
INFO:jmppeft.utils.wbm_relax:# Steps: 24; e_form: P=-0.4498, GT=-0.4538, Δ=0.0039, MAE=0.0174
INFO:jmppeft.utils.wbm_relax:# Steps: 20; e_form: P=-1.4855, GT=-1.4930, Δ=0.0075, MAE=0.0149
INFO:jmppeft.utils.wbm_relax:# Steps: 20; e_form: P=-0.7100, GT=-0.6779, Δ=0.0321, MAE=0.0183
INFO:jmppeft.utils.wbm_relax:# Steps: 22; e_form: P=-1.2129, GT=-1.2189, Δ=0.0059, MAE=0.0163
INFO:jmppeft.utils.wbm_relax:# Steps: 24; e_form: P=-0.2626, GT=-0.2397, Δ=0.0229, MAE=0.0172
INFO:jmppeft.utils.wbm_relax:# Steps: 28; e_form: P=-0.3472, GT=-0.3257, Δ=0.0214, MAE=0.0177
INFO:jmppeft.utils.wbm_relax:# Steps: 24; e_form: P=-0.7818, GT=-0.7816, Δ=0.0002, MAE=0.0158
INFO:jmppeft.utils.wbm_relax:# Steps: 33; e_form: P=-0.2501, GT=-0.2663, Δ=0.0162, MAE=0.0158
INFO:jmppeft.utils.wbm_relax:# Steps: 49; e_form: P=-0.7805, G

2


In [12]:
import dill

x = "mptrj-jmps-ec20.0_fc20.0_sc10.0"
path = f"../relaxresults/{x}.dill"
with open(path, "wb") as f:
    dill.dump(preds_targets, f)